In [ ]:
import json
import logging

import rich
import dotenv

import networkx

In [ ]:
from twon_lss.simulations.twon_base import (
    Simulation,
    SimulationArgs,
    Ranker,
    Agent,
    AgentInstructions,
)

from twon_lss.schemas import Post, User, Feed, Network
from twon_lss.utility import LLM, Message

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
NUM_AGENTS: int = 10
LENGTH_AGENT_MEMORY: int = 10

In [ ]:
ENV = dotenv.dotenv_values("../" * 3 + ".env")
AGENTS_INSTRUCTIONS_CFG = json.load(open("./data/agents.instructions.json"))
AGENTS_PERSONAS_CFG = json.load(open("./data/agents.personas.json"))[:NUM_AGENTS]

rich.print(AGENTS_INSTRUCTIONS_CFG)
rich.print(len(AGENTS_PERSONAS_CFG))

In [ ]:
RANKER = Ranker(
    llm=LLM(api_key=ENV["HF_TOKEN"], model="BAAI/bge-m3", url="https://router.huggingface.co/hf-inference/models/BAAI/bge-m3/pipeline/sentence-similarity"),
)

In [ ]:
# solution for real user name
USERS = [User() for _ in range(NUM_AGENTS)]

In [ ]:
NETWORK = Network.from_graph(networkx.random_regular_graph(4, len(USERS)), USERS)
networkx.draw(NETWORK.root)

In [ ]:
histories = [
    [message.get("content") for message in personas["messages"] if message.get("role") == "assistant"]
    for personas in AGENTS_PERSONAS_CFG
]
histories[0][:2], histories[1][:2]

In [ ]:
FEED = Feed(
    [
        Post(user=user, content=post)
        for user, history in zip(USERS, histories, strict=False)
        for post in history[:2]
    ]
)
len(FEED)

In [ ]:
AGENT_LLM = LLM(api_key=ENV["HF_TOKEN"])

In [ ]:
INDIVIDUALS = {
    user: Agent(
        llm=AGENT_LLM,
        instructions=AgentInstructions(
            persona=AGENTS_INSTRUCTIONS_CFG["persona"], **AGENTS_INSTRUCTIONS_CFG["actions"]
        ),
        memory=history["messages"][1:LENGTH_AGENT_MEMORY + 1],
    )
    for user, history in zip(
        USERS, AGENTS_PERSONAS_CFG, strict=False
    )
}
rich.print(INDIVIDUALS.get(USERS[0]))

In [ ]:
simulation = Simulation(
    args=SimulationArgs(num_steps=2, num_posts_to_interact_with=2),
    ranker=RANKER,
    individuals=INDIVIDUALS,
    network=NETWORK,
    feed=FEED,
)

In [ ]:
simulation()